In [1]:
import pandas as pd 
bookings = pd.read_csv(r'D:\vscode_files\python\notebooks\karpov_course\python\1_pandas\2_bookings.csv', sep = ';')
bookings_head = bookings.head(7)

In [2]:
bookings.shape

(119390, 21)

In [3]:
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [4]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  object 
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  object 
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  object 
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     11

In [5]:
bookings = bookings.rename(columns=lambda col: col.replace(" ", "_").lower())
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [6]:
top_countries = bookings \
    .groupby(['country'], as_index=False) \
    .aggregate({'is_canceled': 'sum'}) \
    .sort_values('is_canceled', ascending=False) \
    .head(6)
top_countries

,country,is_canceled
135,PRT,27519
59,GBR,2453
51,ESP,2177
56,FRA,1934
81,ITA,1333
43,DEU,1218


In [7]:
mean_nights = bookings \
    .groupby(['hotel'], as_index=False) \
    .aggregate({'stays_total_nights': 'mean'}) \
    .sort_values('stays_total_nights', ascending=False) \
    .round(2)
mean_nights

,hotel,stays_total_nights
1,Resort Hotel,4.32
0,City Hotel,2.98


In [8]:
wrong_types_of_rooms = bookings \
    .query('reserved_room_type != assigned_room_type') \
    .shape 
wrong_types_of_rooms

(14917, 21)

In [9]:
bookings.groupby('arrival_date_year').arrival_date_month.agg(pd.Series.mode)

arrival_date_year
2015    September
2016      October
2017          May
Name: arrival_date_month, dtype: object

In [10]:
bookings.groupby(['arrival_date_year', 'arrival_date_month']).is_canceled.sum()

arrival_date_year  arrival_date_month
2015               August                1598
                   December               973
                   July                  1259
                   November               486
                   October               1732
                   September             2094
2016               April                 2061
                   August                1825
                   December              1398
                   February              1337
                   January                557
                   July                  1499
                   June                  2096
                   March                 1477
                   May                   1915
                   November              1636
                   October               2514
                   September             2022
2017               April                 2463
                   August                1816
                   February              1

In [11]:
bookings.groupby(['arrival_date_year', 'arrival_date_month']).is_canceled.sum().groupby(['arrival_date_year']).nlargest(1)


arrival_date_year  arrival_date_year  arrival_date_month
2015               2015               September             2094
2016               2016               October               2514
2017               2017               May                   2762
Name: is_canceled, dtype: int64

In [12]:
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [13]:
bookings['total_kids'] = bookings.children + bookings.babies
bookings.groupby('hotel').total_kids.mean().round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

In [14]:
def has_kids(x):
    if x > 0:
        return True
    else:
        return False

In [15]:
bookings['has_kids'] = bookings.total_kids.apply(has_kids)

In [16]:
bookings['has_kids'].value_counts()

has_kids
False    110058
True       9332
Name: count, dtype: int64

In [17]:
bookings[['has_kids', 'is_canceled']].value_counts() / bookings[['has_kids']].value_counts() * 100

has_kids  is_canceled
False     0              62.778717
          1              37.221283
True      0              65.077154
          1              34.922846
Name: count, dtype: float64